# Notebook #4: Cálculo de Rentabilidad

- Con los datos extraídos y el modelo de *machine learning* que se ha entrenado para predecir el precio del alquiler, somos capaces de realizar el cálculo de rentabilidad con ayuda de distintas funciones.

- Importamos las librerías y soportes.

In [432]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss
from src import soporte_blip as sb
from src import soporte_mongo as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- Como primer paso, definimos una variable con las rutas de los transformers: el encoder, el scaler y el modelo Random Forest.

- También vamos a conectarnos a la base de datos de Mongo para obtener el los datos correspondientes a viviendas en venta.

- Utilizaremos la función `predecir_alquiler` para aplicar el modelo a la totalidad de viviendas en venta, que devuelve el dataframe con una nueva columna 'alquiler_predicho'. Sus únicos argumentos son el archivo pickle con las viviendas en venta y la variable con las rutas de los transformers.

In [492]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
bd = sm.conectar_a_mongo("ProyectoRentabilidad")
gdf = sm.importar_a_geodataframe(bd, "venta")

- Comprobamos el dataframe.

- Para cada vivienda, tenemos una columna que corresponde a una lista con las urls de las imágenes. Para poder identificar cuál corresponde a cada estancia, utilizaremos el modelo YOLOv11, en su versión x y de clasificación, por cuanto no nos interesa determinar la ubicación ni la forma de los objetos en las imágenes, solamente su existencia.

- Con la función `identificar_urls_habitaciones`, se añaden etiquetas a la cocina y al baño de cada anuncio. Para optimizar el tiempo de procesamiento, cuando las ha encontrado, no continua con el resto de imágenes, sino que salta al siguiente anuncio.

- El resultado es un dataframe con nuevas columnas para la URL del baño y de la cocina. Cuando uno de los dos valores es nulo, elimina esa fila. El return también incluye un dataframe con todas las detecciones de las imágenes procesadas, como una forma de poder validar los resultados.

In [ ]:
df, detecciones = sy.identificar_urls_habitaciones(df, 'urls_imagenes', drop_nulls=True)

100%|██████████| 365/365 [55:16<00:00,  9.09s/it] 


- Vamos a comprobar el resultado del filtrado.

- Además del dataframe con las imágenes filtradas, tenemos un segundo dataframe como retorno, que almacena los elementos que se han detectado en cada imagen, la URL y si ha sido etiquetado como cocina o baño. Esto nos permite realizar generar una métrica sobre la efectividad de YOLO.

- De las 365 viviendas en venta, se han encontrado imágenes de baño y cocina para 278. Dado que el cliente objetivo del proyecto no es quién se dedica a reformas, sin poder conocer el estado de estas estancias, el cálculo de rentabilidad puede cambiar significativamente por ser los espacios que cuesta más dinero reformar.

- Por ese motivo, las eliminaremos del conjunto de datos con el parámetro *drop nulls*.

In [324]:
df.shape[0]

278

In [329]:
detecciones[detecciones['habitación']=='kitchen'].head(5)

,url,detecciones,habitación
2,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, refrigerator, wardrobe, dishwasher...",kitchen
7,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washbasin, refrigerator, washer, s...",kitchen
13,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washer, stove, dishwasher, washbasin]",kitchen
14,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, washbasin, refrigerator, s...",kitchen
16,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, space_heater, dishwasher, ...",kitchen


- Al comprobar las URLs de forma manual utilizando una muestra aleatoria, se confirma una accuracy de entorno al 90% en el etiquetado.

In [333]:
for value in detecciones[detecciones['habitación']=='kitchen']['url'].sample(10):
    print(value)

https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/56/4e/c7/1223985031.webp
https://img4.idealista.com/blur/WEB_LISTING-M/90/id.pro.es.image.master/83/d6/88/1214278166.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/13/72/5b/1292527697.webp
https://img4.idealista.com/blur/WEB_LISTING-M/90/id.pro.es.image.master/5b/07/05/1285184777.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/6e/25/4d/1274779622.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/8a/ae/e9/1247197620.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/2e/20/c7/1289354886.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/3e/80/3f/1277556572.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/91/d5/e9/1129570083.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/a1/e0/16/1281241400.webp


- Como una capa de verificación adicional, vamos a utilizar el modelo BLIP de Salesforce, que crea descripciones de las imágenes, en este caso, de las URLs identificadas por YOLO. Contamos luego el número de ocurrencias de 'kitchen' o 'bathroom'.

In [320]:
captions_cocinas = sb.generar_descripciones(df, 'url_cocina')
sb.contar_palabras(captions_cocinas, 'kitchen')

278it [03:44,  1.24it/s]


245

In [323]:
captions_banios = sb.generar_descripciones(df, 'url_banio')
sb.contar_palabras(captions_banios, 'bathroom')

278it [03:26,  1.35it/s]


261

- De las 278 imágenes identificadas como cocinas por YOLO, BLIP ha coincidido en 245, un 88%. El dato en el caso de los baños mejora, habiendo encontrado 261 coincidencias, un 94%.

- Conocedores de que, habrá falsos negativos y falsos positivos en ambos modelos, el alto porcentaje de coincidencias en ambos casos nos indica que la fiabilidad del primer cribado es alta.

- Una observación adicional a lo detectado en el EDA es que, aquellos anuncios en que hemos rellenado el valor nulo del anunciante con 'ND', corresponden generalmente a particulares, lo cual es relevante porque podría significar entorno a un 3% de ahorro en el valor de compra.

In [314]:
df[df['anunciante']=='ND'].sample(2)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio
87,106652992,89000.0,1780.0,piso,True,1,False,50.0,2,1,True,False,False,False,good,"calle Legión Gemina, 11",OPORTUNIDAD que no te puedes perder! PISO en v...,ND,608197088,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.92042 41.65292),Oliver-Valdefierro,835.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
90,35110384,149000.0,1713.0,piso,True,1,False,87.0,3,1,False,True,True,False,good,"calle Goya, 10","Reforma integral: eléctricidad, tuberías sanit...",ND,625675251,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-1.0289 41.7224),Distrito Rural,891.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...


- El siguiente paso será utilizar la API de Anthropic, y específicamente el modelo Sonnet, para asignar una puntuación del 1 al 10 al estado de la estancia, y una estimación de los metros cuadrados, que es una consideración que ha surgido durante el desarrollo y la fase de pruebas del proyecto.

- El prompt que se utiliza incluye:

<div style="background-color:rgb(149, 222, 237); padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
You are an AI image analysis system specialized in evaluating property conditions. Your task is to analyze multiple property images in batch and provide a precise evaluation.
Instructions for Analysis:

Analyze each pair of images:

Image 1: Kitchen
Image 2: Bathroom

Evaluation Criteria:

Ratings: Whole numbers 1-5
1: Very poor (complete renovation required)
2: Poor (major renovations needed)
3: Fair (some renovations needed)
4: Good (minor improvements required)
5: Excellent (no renovations needed)

Sizes: Whole numbers in square meters (m²)
</div>

- Como puede verse en el prompt, el resultado de la consulta son 4 valores separados por comas, con la función `analizar_propiedades`, separaremos esos resultados y los asignaremos a 4 nuevas columnas del dataframe. Para tener consultas más eficientes, se enviarán en lotes, o *batches*, que corresponden a grupos de viviendas. Este batch más el dataframe con las URLs identificadas son los parámetros de la función.

In [138]:
df_sample = df.head(3)

In [186]:
df_sample, resultados = ss.analizar_propiedades(df_sample, batch=3)

Procesando lotes:   0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
df_sample

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,4
1,106086132,139900.0,1686.0,piso,True,3,True,83.0,2,1,False,False,True,False,good,calle de Portugal,Financiación hipoteca 100% ¡oportunidad única ...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.90232 41.65202),Delicias,761.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,4
2,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.87 41.69848),Distrito Rural,1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,12,4


In [334]:
df.to_pickle("../data/transformed/final_tags.pkl")

In [335]:
df_importado = pd.read_pickle("../data/transformed/final_tags.pkl")
df_importado.shape[0]

278

In [431]:
sr.calcular_rentabilidad_inmobiliaria_wrapper(df, porcentaje_entrada=0.2, coste_reformas=10000, comision_agencia=5000, anios=30, tin=0.03, seguro_vida=0, tipo_irpf=0.37, porcentaje_amortizacion=0.4)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio,Coste Total,Rentabilidad Bruta,Beneficio Antes de Impuestos,Rentabilidad Neta,Cuota Mensual Hipoteca,Cash Necesario Compra,Cash Total Compra y Reforma,Beneficio Neto,Cashflow Antes de Impuestos,Cashflow Después de Impuestos,ROCE,ROCE (Años),Cash-on-Cash Return,COCR (Años)
200,106848232,92000.0,989.0,piso,True,2,False,93.0,2,2,True,False,False,False,good,calle de Basilio Boggiero,Findliving te presenta esta magnifica vivienda...,Findliving Real Estate,876210931,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.88601 41.65536),Casco Histórico,1233.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,116200.0,12.73,9882.87,6.97,-310.30,32600.0,37600.0,8095.13,7429.54,5641.80,34.73,2.88,13.24,7.55
166,106739190,74000.0,379.0,casa de campo,ND,ND,ND,195.0,1,1,False,True,True,True,good,"camino Cubero, 24",MUNDOCASA te ofrece en exclusiva este Terreno ...,Mundocasa Las Fuentes,876210750,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.78742 41.75831),Distrito Rural,920.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,96400.0,11.45,7161.89,6.14,-249.59,27200.0,32200.0,5918.27,5188.56,3944.94,29.68,3.37,10.60,9.43
231,106632479,65000.0,1300.0,piso,True,bj,False,50.0,2,1,False,False,False,False,good,calle de Pedro Cubero,Propiedades Zaragoza lanza esta increible vivi...,Propiedades Zaragoza,876210746,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86073 41.64935),Las Fuentes,822.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,86500.0,11.40,6370.02,6.10,-219.23,24500.0,29500.0,5272.22,4636.69,3538.89,28.59,3.50,10.26,9.75
29,104860465,73000.0,608.0,casa de campo,ND,ND,ND,120.0,3,1,False,True,False,True,good,camping bohalar,Descubre esta increíble oportunidad en la mejo...,ESPAZIOS,876210119,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-1.06841 41.64826),Distrito Rural,884.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,95300.0,11.13,6829.82,5.94,-246.22,26900.0,31900.0,5656.59,4883.16,3709.93,28.75,3.48,10.05,9.95
271,106077006,118000.0,1054.0,piso,True,2,False,112.0,4,2,False,False,False,False,good,"calle de Basilio Boggiero, 72","Precioso y luminoso piso de 4 habitaciones, to...",ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.88889 41.65587),Casco Histórico,1334.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,144800.0,11.06,10400.38,5.93,-397.99,40400.0,45400.0,8584.34,7253.71,5437.67,31.76,3.15,10.79,9.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,99403051,139500.0,2051.0,piso,True,bj,False,68.0,2,1,False,False,True,False,good,calle de la Madre Sacramento,Vivienda con terraza a la venta en la Calle Ma...,Espazio ZG,876210097,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.89436 41.64904),Delicias,704.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,168450.0,5.02,3892.91,2.13,-470.51,46850.0,51850.0,3593.12,172.91,-126.88,14.86,6.73,-0.22,-448.06
130,106725265,146500.0,2187.0,piso,True,2,True,67.0,3,1,False,False,False,False,good,avenida de Navarra,"¿Buscas un piso amplio, reformado y con todas ...",Doctor Property,876210316,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.90184 41.65454),Delicias,730.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,176150.0,4.97,4020.65,2.11,-494.12,48950.0,53950.0,3715.81,113.99,-190.85,1